In [36]:
import numpy as np
import time

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose

In [38]:
transform = Compose([ToTensor()])

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

In [12]:
# Convert images to numpy arrays and flatten them
X_train = training_data.data.numpy().reshape(len(training_data), -1) / 255.0
y_train = training_data.targets.numpy()

X_test = test_data.data.numpy().reshape(len(test_data), -1) / 255.0
y_test = test_data.targets.numpy()

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (60000, 784)
Test shape: (10000, 784)


In [34]:
def optimize_and_evaluate(pipeline, param_grid, name):
    print("=" * 80)
    print(f"Optimizing {name} with 10-fold cross-validation...")

    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=10,
        n_jobs=-1,
        verbose=1
    )

    start_time = time.time()
    grid.fit(X_train, y_train)
    cv_time = time.time() - start_time

    print(f"\nBest parameters for {name}:")
    print(grid.best_params_)

    print(f"\nBest CV accuracy ({name}): {grid.best_score_:.4f}")
    print(f"CV training time ({name}): {cv_time:.2f} seconds")

    # Evaluate on the test set
    best_model = grid.best_estimator_

    start_time = time.time()
    predictions = best_model.predict(X_test)
    pred_time = time.time() - start_time

    print("\nTest Accuracy:", accuracy_score(y_test, predictions))
    print("\nClassification Report:")
    print(classification_report(y_test, predictions))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print()

    print(f"Test prediction time ({name}): {pred_time:.2f} seconds\n")

In [40]:
linear_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=100)),
    ("svm", LinearSVC(dual=False, max_iter=10000))
])
linear_params = {"svm__C": [0.01, 0.1, 1, 10, 100]}

poly_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=100)),
    ("svm", SVC(kernel="poly", degree=2))
])
poly_params = {
    "svm__C": [0.01, 0.1, 1, 10],
    "svm__coef0": [0, 0.5, 1],
    "svm__gamma": ["scale", "auto"]
}

rbf_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=100)),
    ("svm", SVC(kernel="rbf"))
])
rbf_params = {
    "svm__C": [0.1, 1, 10],
    "svm__gamma": ["scale", "auto"]
}

In [39]:
optimize_and_evaluate(linear_pipeline, linear_params, "Linear SVM")

Optimizing Linear SVM with 10-fold cross-validation...
Fitting 10 folds for each of 5 candidates, totalling 50 fits

Best parameters for Linear SVM:
{'svm__C': 100}

Best CV accuracy (Linear SVM): 0.8416
CV training time (Linear SVM): 757.04 seconds

Test Accuracy: 0.8299

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.80      0.79      1000
           1       0.97      0.95      0.96      1000
           2       0.72      0.71      0.72      1000
           3       0.80      0.87      0.83      1000
           4       0.70      0.76      0.73      1000
           5       0.93      0.91      0.92      1000
           6       0.62      0.48      0.54      1000
           7       0.90      0.93      0.91      1000
           8       0.92      0.94      0.93      1000
           9       0.94      0.94      0.94      1000

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000

In [41]:
optimize_and_evaluate(poly_pipeline, poly_params, "Polynomial SVM (degree 2)")
optimize_and_evaluate(rbf_pipeline, rbf_params, "RBF SVM")

Optimizing Polynomial SVM (degree 2) with 10-fold cross-validation...
Fitting 10 folds for each of 24 candidates, totalling 240 fits

Best parameters for Polynomial SVM (degree 2):
{'svm__C': 10, 'svm__coef0': 0.5, 'svm__gamma': 'scale'}

Best CV accuracy (Polynomial SVM (degree 2)): 0.8957
CV training time (Polynomial SVM (degree 2)): 5154.86 seconds

Test Accuracy: 0.8877

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.86      0.84      1000
           1       0.99      0.97      0.98      1000
           2       0.80      0.81      0.80      1000
           3       0.88      0.90      0.89      1000
           4       0.81      0.83      0.82      1000
           5       0.97      0.95      0.96      1000
           6       0.72      0.68      0.70      1000
           7       0.93      0.96      0.95      1000
           8       0.98      0.97      0.97      1000
           9       0.97      0.95      0.96      1000

   